<a href="https://colab.research.google.com/github/takzen/pytorch-black-belt/blob/main/notebooks/15_Dataset_vs_IterableDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --------------------------------------------------------------
# ☁️ COLAB SETUP (Automatyczna instalacja środowiska)
# --------------------------------------------------------------
import sys
import os

# Sprawdzamy, czy jesteśmy w Google Colab
if 'google.colab' in sys.modules:
    print('☁️ Wykryto środowisko Google Colab. Konfiguruję...')

    # 1. Pobieramy plik requirements.txt bezpośrednio z repozytorium
    !wget -q https://raw.githubusercontent.com/takzen/ai-engineering-handbook/main/requirements.txt -O requirements.txt

    # 2. Instalujemy biblioteki
    print('⏳ Instaluję zależności (to może chwilę potrwać)...')
    !pip install -q -r requirements.txt

    print('✅ Gotowe! Środowisko jest zgodne z repozytorium.')
else:
    print('💻 Wykryto środowisko lokalne. Zakładam, że masz już uv/venv.')


# 🥋 Lekcja 15: Dataset vs IterableDataset (Streaming danych)

W PyTorch mamy dwa sposoby na karmienie modelu danymi:

1.  **Map-style (`Dataset`):**
    *   Musisz znać długość (`__len__`).
    *   Musisz mieć dostęp do każdego elementu (`__getitem__(idx)`).
    *   *Idealne do:* Zdjęć na dysku, małych plików CSV.

2.  **Iterable-style (`IterableDataset`):**
    *   Działa jak strumień (Generator).
    *   Nie musi znać końca danych.
    *   *Idealne do:* Petabajtów tekstu, streamingu z sieci, logów serwera.

W tej lekcji napiszemy **poprawną klasę `IterableDataset`**, która potrafi bezpiecznie dzielić pracę, nawet jeśli uruchomimy ją na wielu workerach (na serwerze produkcyjnym).

In [7]:
import torch
from torch.utils.data import Dataset, IterableDataset, DataLoader
import math

# Symulacja danych (np. linie w ogromnym pliku tekstowym)
data_source = list(range(20))

print(f"Dane źródłowe: {data_source}")

Dane źródłowe: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


## Podejście 1: Klasyczny Map-style (Standard)

To znasz. Proste i skuteczne, ale wymaga załadowania indeksów do pamięci.

In [8]:
class MyMapDataset(Dataset):
    def __init__(self, data):
        self.data = data
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

# Test
map_ds = MyMapDataset(data_source)
loader = DataLoader(map_ds, batch_size=4, shuffle=True)

print("--- Map Dataset (Działa losowo) ---")
for batch in loader:
    print(batch.tolist())

--- Map Dataset (Działa losowo) ---
[16, 13, 11, 7]
[5, 0, 12, 3]
[10, 14, 19, 4]
[9, 17, 8, 6]
[1, 18, 2, 15]


## Podejście 2: IterableDataset (Streaming)

Tutaj implementujemy metodę `__iter__`.

**Kluczowy mechanizm (Workload Splitting):**
Jeśli uruchomimy to na wielu procesorach (workerach), każdy dostanie kopię datasetu.
Musimy ręcznie sprawdzić `get_worker_info()`, żeby każdy worker wziął **inny kawałek tortu**.
Inaczej model uczyłby się na zduplikowanych danych.

*Poniższy kod jest "Production Ready" - zadziała poprawnie zarówno na 1 procesie (Windows/Jupyter), jak i na 100 procesach (Klaster Linux).*

In [9]:
class SmartIterableDataset(IterableDataset):
    def __init__(self, data):
        self.data = data
        
    def __iter__(self):
        worker_info = torch.utils.data.get_worker_info()
        
        if worker_info is None:
            # SCENARIUSZ A: Jeden proces (np. Jupyter na Windowsie)
            # Bierzemy całe dane od początku do końca.
            iter_start = 0
            iter_end = len(self.data)
            iter_step = 1
        else:
            # SCENARIUSZ B: Wiele workerów (np. Serwer treningowy)
            # Dzielimy dane, żeby workery nie dublowały pracy.
            worker_id = worker_info.id
            num_workers = worker_info.num_workers
            
            # Każdy worker bierze co n-ty element (np. co 4)
            iter_start = worker_id
            iter_end = len(self.data)
            iter_step = num_workers
            
        # Generator (yield) - zwraca dane kawałek po kawałku
        for i in range(iter_start, iter_end, iter_step):
            yield self.data[i]

print("Klasa zdefiniowana. Gotowa do użycia.")

Klasa zdefiniowana. Gotowa do użycia.


## Uruchomienie (Bezpieczne dla Windows)

Użyjemy `num_workers=0`.
Dlaczego? Bo Jupyter na Windowsie nie obsługuje wieloprocesowości dla klas zdefiniowanych wewnątrz komórki.
Ale dzięki naszej logice `if worker_info is None`, kod zadziała bezbłędnie i przetworzy wszystkie dane.

In [10]:
# Inicjalizacja
iter_ds = SmartIterableDataset(data_source)

# Tworzymy Loader (num_workers=0 zapewnia stabilność w notatniku)
loader = DataLoader(iter_ds, batch_size=4, num_workers=0)

print("--- Iterable Dataset (Streaming) ---")
all_data = []

for batch in loader:
    print(f"Batch: {batch.tolist()}")
    all_data.extend(batch.tolist())

print("-" * 30)
print(f"Odebrano łącznie: {len(all_data)} elementów.")
# Sprawdzenie poprawności
if sorted(all_data) == data_source:
    print("✅ SUKCES: Wszystkie dane zostały przetworzone poprawnie (bez duplikatów).")
else:
    print("❌ BŁĄD: Coś się zgubiło lub zdublowało.")

--- Iterable Dataset (Streaming) ---
Batch: [0, 1, 2, 3]
Batch: [4, 5, 6, 7]
Batch: [8, 9, 10, 11]
Batch: [12, 13, 14, 15]
Batch: [16, 17, 18, 19]
------------------------------
Odebrano łącznie: 20 elementów.
✅ SUKCES: Wszystkie dane zostały przetworzone poprawnie (bez duplikatów).


## 🥋 Black Belt Summary

1.  **IterableDataset** to konieczność przy Big Data (gdy nie możesz zrobić `len(data)`).
2.  **Pułapka Duplikatów:** Domyślnie PyTorch kopiuje dataset do każdego workera. Musisz użyć `get_worker_info()` wewnątrz `__iter__`, żeby podzielić pracę.
3.  **Shuffle:** W `IterableDataset` nie ma globalnego tasowania (`shuffle=True` nie zadziała idealnie). Tasuje się tylko lokalnie w buforze (o czym więcej w module zaawansowanym).